In [1]:
import duckdb
import pandas as pd

%load_ext sql

In [2]:
# %sql duckdb:///:default:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [3]:
# %config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)


In [4]:
%%sql
CREATE SCHEMA dannys_diner;

SET search_path = dannys_diner;

CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');

    
CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');
    
CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');

Count
2


### Table Information 
#### There are 3 tables in the dataset : 
#### 1.  Sales table contains customer_id, order_date (without timestamp) and product_id
#### 2.  Menu table contains product_id, product_name and price 
#### 3.  Members table contains customer_id and join_date(without timestamp)

#### Displayed below each individual table: 

In [115]:
%%sql

    SELECT *
      FROM dannys_diner.sales;

customer_id,order_date,product_id
A,2021-01-01,1
A,2021-01-01,2
A,2021-01-07,2
A,2021-01-10,3
A,2021-01-11,3
A,2021-01-11,3
B,2021-01-01,2
B,2021-01-02,2
B,2021-01-04,1
B,2021-01-11,1


In [116]:
%%sql

    SELECT *
      FROM dannys_diner.menu;

product_id,product_name,price
1,sushi,10
2,curry,15
3,ramen,12


In [117]:
%%sql

    SELECT *
      FROM dannys_diner.members;

customer_id,join_date
A,2021-01-07
B,2021-01-09


### Table Join Keys 
#### Between Sales Table and Menu Table, the join key is product_id
#### Between Sales Table and Members Table, the join key is customer_id
#### Between Menu Table and Members Table, the join key is customer_id

### Case Study Questions

#### 1. What is the total amount each customer spent at the restaurant?
#### Solution :

In [30]:
%%sql

    SELECT ds.customer_id,
           SUM(dm.price) AS total_spent
      FROM dannys_diner.menu AS dm
      JOIN dannys_diner.sales AS ds
        ON dm.product_id=ds.product_id
     GROUP BY ds.customer_id
     ORDER BY ds.customer_id;

customer_id,total_spent
A,76
B,74
C,36


#### 2. How many days has each customer visited the restaurant?
#### Solution :

In [113]:
%%sql

    SELECT customer_id,
           COUNT(order_date) AS total_visit
      FROM dannys_diner.sales 
     GROUP BY customer_id
     ORDER BY customer_id;
    

customer_id,total_visit
A,6
B,6
C,3


In [114]:
%%sql

    SELECT customer_id,
           COUNT(distinct order_date) as visit_days
      FROM dannys_diner.sales
     GROUP BY customer_id
     ORDER BY customer_id;
    

customer_id,visit_days
A,4
B,6
C,2


#### 3. What was the first item from the menu purchased by each customer?
#### Solution :

In [9]:
%%sql

WITH first_order AS (
    
                    SELECT *
                      FROM (SELECT  customer_id,
                                    product_id,
                                    order_date,
                                    ROW_NUMBER()OVER(PARTITION BY customer_id ORDER BY order_date) AS rank
                              FROM  dannys_diner.sales
                             ORDER  BY customer_id,order_date)t
                      WHERE rank=1)
 
     SELECT customer_id,
            fo.product_id,
            dm.product_name,
            order_date
       FROM first_order AS fo
       JOIN dannys_diner.menu AS dm 
         ON fo.product_id=dm.product_id;


customer_id,product_id,product_name,order_date
A,1,sushi,2021-01-01
B,2,curry,2021-01-01
C,3,ramen,2021-01-01


#### 4. What is the most purchased item on the menu and how many times was it purchased by all customers?
#### Solution :

In [10]:
%%sql

    SELECT ds.product_id,
           COUNT(ds.product_id) AS total_order,
           dm.product_name
      FROM dannys_diner.sales AS ds
      JOIN dannys_diner.menu AS dm
        ON ds.product_id=dm.product_id
     GROUP BY ds.product_id,dm.product_name;
    

product_id,total_order,product_name
1,3,sushi
2,4,curry
3,8,ramen


#### 5. Which item was the most popular for each customer?
#### Solution :

In [66]:
%%sql

WITH base AS (
    
             SELECT  customer_id,
                     product_id, 
                     count(*) as purchase_count
               FROM  dannys_diner.sales
              GROUP  BY customer_id, product_id
),

     maxi AS (
         
            SELECT  *,
                    MAX(purchase_count) OVER (PARTITION BY customer_id) AS max_purchase
              FROM  base
             ORDER  BY customer_id
)

    SELECT  mi.customer_id,
            mi.product_id,
            m.product_name,
            purchase_count
      FROM  maxi mi
      JOIN  annys_diner.menu m
        ON  mi.product_id = m.product_id
     WHERE  purchase_count = max_purchase;
    

customer_id,product_id,product_name,purchase_count
A,3,ramen,3
B,2,curry,2
B,1,sushi,2
B,3,ramen,2
C,3,ramen,3


#### Note : For customer B, all products are bought twice (equally frequent). Thus all 3 items are the most popluar for customer B.

#### 6. Which item was purchased first by the customer after they became a member?
#### Solution :

In [79]:
%%sql

WITH base AS (
    
            SELECT *
              FROM dannys_diner.sales sal
              JOIN dannys_diner.members mem
                ON sal.customer_id = mem.customer_id
             WHERE order_date >= join_date
), 

rank AS (
            SELECT *,
                   ROW_NUMBER()OVER(PARTITION BY customer_id ORDER BY order_date) as purchase_rank
              FROM base
)

    SELECT customer_id, 
           join_date,
           order_date,
           product_name
      FROM rank r
      JOIN dannys_diner.menu m
        ON r.product_id = m.product_id
     WHERE purchase_rank = 1;
    

customer_id,join_date,order_date,product_name
A,2021-01-07,2021-01-07,curry
B,2021-01-09,2021-01-11,sushi


#### Note : Why is customer C not in the table? This is because just a regular customer, not a member.Therefore there is no join_date for Customer C, only order_date is available.

#### 7. Which item was purchased just before the customer became a member?
#### Solution :

In [88]:
%%sql

WITH base AS (
    
            SELECT *
              FROM dannys_diner.sales sal
              JOIN dannys_diner.members mem
                ON sal.customer_id = mem.customer_id
             WHERE order_date < join_date
),

     net AS (
         
            SELECT *,
                   ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY order_date) as purchase_rank,
                   COUNT()OVER(PARTITION BY  customer_id) as max_purchase
              FROM base
)

    SELECT customer_id,
           join_date,
           order_date,
           n.product_id,
           product_name
      FROM net n
      JOIN dannys_diner.menu m
        ON n.product_id = m.product_id
     WHERE purchase_rank = max_purchase;
    

customer_id,join_date,order_date,product_id,product_name
A,2021-01-07,2021-01-01,2,curry
B,2021-01-09,2021-01-04,1,sushi


#### 8. What is the total items and amount spent for each member before they became a member?
#### Solution :

In [118]:
%%sql

    SELECT s.customer_id,
           COUNT(s.product_id) AS total_items,
           SUM(price) as amount_spent
      FROM dannys_diner.sales s
      JOIN dannys_diner.members m
        ON m.customer_id = s.customer_id AND order_date < join_date
      JOIN dannys_diner.menu men
        ON s.product_id = men.product_id
     GROUP BY s.customer_id
     ORDER BY s.customer_id;
        

customer_id,total_items,amount_spent
A,2,25
B,3,40


#### 9. If each 1 dollar spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
#### Solution :

In [119]:
%%sql

    SELECT customer_id,
           SUM(CASE 
               WHEN product_name = 'sushi' THEN price*2*10
               ELSE price*10 
               END) AS points
      FROM dannys_diner.sales s
      JOIN dannys_diner.menu m
        ON s.product_id = m.product_id
     GROUP BY customer_id;
    

customer_id,points
A,860
B,940
C,360


#### 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?
#### Solution :

In [111]:
%%sql

WITH base AS (
    
            SELECT s.customer_id,
                   (CASE
                    WHEN s.order_date < mem.join_date + 7 AND s.order_date >= mem.join_date
                    THEN 'first_week'
                    ELSE null 
                    END) as first_week,
                   (CASE 
                    WHEN first_week IS NOT null THEN price*2*10
                    WHEN first_week IS null and product_name = 'sushi' THEN price*2*10
                    WHEN first_week IS null THEN price*10
                    ELSE null 
                    END) AS points
             FROM  dannys_diner.sales s
             JOIN  dannys_diner.menu m
               ON  s.product_id = m.product_id
        LEFT JOIN  dannys_diner.members mem
               ON  s.customer_id = mem.customer_id
            WHERE  order_date < '2021-02-01'
)

            SELECT customer_id,
                   SUM(points)
              FROM base
             GROUP BY customer_id;
        

customer_id,sum(points)
A,1370
B,820
C,360


In [20]:
%%sql 

WITH table_1 AS(
     SELECT ds.customer_id,
            dm.join_date,
            ds.order_date,
            ds.product_id,
            (CASE 
             WHEN ds.product_id IN(1,2,3) THEN dme.price*20
             ELSE 0
             END) AS total_points_first_week
       FROM dannys_diner.sales AS ds
       JOIN dannys_diner.members AS dm
         ON ds.customer_id=dm.customer_id AND ds.order_date BETWEEN dm.join_date AND dm.join_date+7
       JOIN dannys_diner.menu AS dme
         ON ds.product_id=dme.product_id
      GROUP BY ds.customer_id,dm.join_date,ds.order_date,ds.product_id,dme.price
      ORDER BY ds.customer_id)

     SELECT t1.customer_id,SUM(total_points_first_week) AS total_points_1
       FROM table_1 AS t1
      GROUP BY t1.customer_id;


customer_id,total_points_1
A,780
B,440
